# Transformer - 기본 과제
> 주어진 문장 다음 단어를 예측하는 모델을 만들어본다
   


### RNN vs Transformer
| 항목             | RNN (Recurrent Neural Network)                             | Transformer                                               |
|:------------------|-------------------------------------------------------------|------------------------------------------------------------|
| **기본 구조**     | 순차적으로 시퀀스를 처리 (시간 순서대로)                  | 전체 시퀀스를 동시에 처리 (병렬처리 가능)                  |
| **병렬 처리**     | ❌ 불가능 (이전 단계 결과가 다음 단계 입력에 필요)         | ✅ 가능 (Self-Attention으로 모든 위치를 동시에 참조)       |
| **장기 의존성 처리** | ❌ 어려움 (Vanishing Gradient 문제)                      | ✅ 훨씬 강력함 (모든 단어 간 관계 파악 가능)               |
| **입력 위치 정보** | 자연스럽게 순서를 따름                                    | 별도로 Positional Encoding 필요                           |
| **대표 모델**     | LSTM, GRU                                                  | BERT, GPT, T5, etc.                                       |
| **성능 및 속도**  | 긴 문장 처리에 약함 / 느림                                 | 긴 문장도 잘 처리 / 빠름 (GPU 병렬화)                      |


   
### IMDb 감성 분류 데이터셋
- Internet Movie Database
- 감성 분류(Sentiment Classification)
  - 영화에 대한 정보 / 사용자 리뷰(text) / 별점
  - 입력: 사용자 리뷰(text)
  - 출력: 감성 라벨(긍정 or 부정)
- 텍스트 분류(Binary Classification) 문제

**Example**
``` makefile
입력: "This movie was surprisingly good and well-acted."
출력: 1 (긍정)
```

``` makefile
입력: "I wasted two hours of my life watching this."
출력: 0 (부정)
```

### import 목록
- `datasets`: Hugging Face의 데이터셋 로더 라이브러리 / IMDb, MNIST 등 다양한 데이터셋 활용 가능
- `sacremoses`: 텍스트 전처리 모듈 / tokenizer 내부에서 사용할 수 있음


In [1]:
!pip install datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" 

## Transformer 구조

### 1️⃣Embedding
[![Notebook](https://img.shields.io/badge/Github-Preview-black?logo=github)](https://github.com/zerovodka/ML-learning/blob/master/src/week2/Embedding.ipynb)
- Embedding process 정리
- BERT / SBERT 모델 비교

### 2️⃣Positional Encoding   
[![Notebook](https://img.shields.io/badge/Github-Preview-black?logo=github)](https://github.com/zerovodka/ML-learning/blob/master/src/week2/Positional-Encoding.ipynb)
- Positional Encoding 수식 정리
- 시각화

### 3️⃣Encoder   
- **Self-Attention** (Multi-Head Attention)
- Feed Forward

### 4️⃣Decoder  
- **`Masked`** **Self-Attention** (**`Masked`** Multi-Head Attention)
- Encoder-Decoder Attention (Multi-Head Attention / Traditional Attention)
- Feed Forward

### 5️⃣Prediction   

# **Attention is all you need**   
![Attention is all you need](https://binarymindset.com/wp-content/uploads/2023/08/transformers-4-574x1024.png)   


### 1️⃣Embedding

**BERT vs GPT2 Tokenizer | AutoTokenizer** 정리  
[![Notebook](https://img.shields.io/badge/Github-Preview-black?logo=github)](https://github.com/zerovodka/ML-learning/blob/master/src/week2/BERT-vs-GPT-Tokenizer.ipynb)
- Emoji 처리 관점에서 비교
- AutoTokenizer
- 실무 vs 실험 적절한 import
- tokenizer 내부 성분 값 설명


In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

**Load BERT Tokenizer**   
- BERT Tokenizer Model의 사전 학습된 구조, 설정, vocabulary를 확인할 수 있다

In [3]:
# ds = load_dataset("stanfordnlp/imdb")

# imdb 데이터 앞에서 5%만 잘라서 수행: 빠른 실험을 위함
train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

print(tokenizer)

print(f'사용중인 tokenizer 모델: \n{tokenizer.name_or_path}')
print(f'\n\n{tokenizer.name_or_path} 모델의 vacabulary size: \n{tokenizer.vocab_size}')
print(f'\n\n{tokenizer.name_or_path} 모델의 max length: \n{tokenizer.model_max_length}')
print(f'\n\n{tokenizer.name_or_path} 모델의 special_tokens_map: \n{tokenizer.special_tokens_map}')
print(f'\n\n{tokenizer.name_or_path} 모델의 decode 시 특수 token 처리 방식: \n{tokenizer.added_tokens_decoder}')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
사용중인 tokenizer 모델: 
bert-base-uncased


bert-base-uncased 모델의 vacabulary size: 
30522


bert-base-uncased 모델의 max length: 
512

**Encode function**
- tokenizer 모델 마다 설정 값이 다르기때문에, 각각에 맞는 encode 함수를 선언한다

**기능**
- tokenizing: `text` -> token 쪼개기
- encoding: token -> number ID 인코딩
- padding / truncation: 패딩, 길이 잘라내기(max length 선언 기준)
- return: 딕셔너리 리턴
  - `return_tensor='pt'` 속성 추가 시: python tensor 리턴   

|옵션|설명|
|:---|:---|
|**`return_tensors='pt'`**|	PyTorch 텐서로 리턴 → 모델 입력에 바로 사용 가능|
|**`return_tensors='tf'`**|	TensorFlow용|
|**`return_tensors='np'`**|	NumPy용
|**`return_tensors=None`**|	파이썬 dict (list 형태) → 디버깅/출력 확인에 유리

In [4]:
# is_tensor 값을 넘기지 않으면, LongTensor 처리를 외부에서 따로 해줘야한다
def encode_bert(text, max_len, is_tensor=False):
  return tokenizer(text, padding=True, truncation=True, max_length=max_len, return_tensors= 'pt' if is_tensor else None)

text = 'This movie was incredibly touching and had great performances!! I would definitely recommend it to anyone 🤔🤔.'

print(f'딕셔너리 형태:\n {encode_bert(text, 30)}')
print(f'PyTorch Tensor 형태:\n {encode_bert(text, 30, True)}')

# Tokenizer의 return 타입에 따라 input_ids, token_type_ids, attention_mask의 type이 다르다
# input_ids: input 값이 토큰화 된 것
# attention_mask: 실제 단어: 1 / padding: 0

딕셔너리 형태:
 {'input_ids': [101, 2023, 3185, 2001, 11757, 7244, 1998, 2018, 2307, 4616, 999, 999, 1045, 2052, 5791, 16755, 2009, 2000, 3087, 100, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
PyTorch Tensor 형태:
 {'input_ids': tensor([[  101,  2023,  3185,  2001, 11757,  7244,  1998,  2018,  2307,  4616,
           999,   999,  1045,  2052,  5791, 16755,  2009,  2000,  3087,   100,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### 배치 데이터 정리 함수
- DataLoader가 train_ds에서 mini_batch 크기 만큼 샘플을 뽑는다
- collate_fn(batch)에 뽑은 샘플을 전달
- collate_fn
  - text 추출 : PyTorch Tensor 형태로 리턴 받음
  - label 추출 : LongTensor를 통해 PyTorch Tensor로 추출한 label type 변경
- (texts, label) 튜플 반환
  - `train_loader`
  - `test_loader`

In [5]:
def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # texts = torch.LongTensor(encode_bert(texts, max_len).input_ids)
  # labels = torch.LongTensor(labels)

  # encode_bert 함수에서 PyTorch Tensor type으로 바로 return했기에, LongTensor 처리가 불필요하다
  texts = encode_bert(texts, max_len, True).input_ids
  labels = torch.LongTensor(labels)

  return texts, labels

mini_batch=64

train_loader = DataLoader(train_ds, mini_batch, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, mini_batch, shuffle=False, collate_fn=collate_fn)

# for inputs, labels in train_loader:
#   print(inputs.shape)
#   print(labels.shape)
#   print(inputs)
#   print(labels)
#   break

# train_loader 내부를 한 배치만 보기
inputs, labels = next(iter(train_loader))
print(f'input size: {inputs.shape}\n')
print(f'label size: {labels.shape}\n')
print(f'input[0]: {inputs[0]}\n')
print(f'decode input[0]:\n {tokenizer.decode(inputs[0])}')
print(f"emotion of decode input[0]:\n {'부정' if labels[0].item() == 0 else '긍정'}")

input size: torch.Size([64, 400])

label size: torch.Size([64])

input[0]: tensor([  101, 27594,  2121,  1024,  1996,  2402,  7089,  1010, 24401,  1010,
         2003,  6476,  2041,  2011,  1996, 23371,  3334,  1010,  5736,  1006,
         1998,  2666, 25005,  1007,  1010,  2138,  2016, 29116,  7164,  2008,
        24401,  2003,  2383,  2019,  6771,  2007,  2028,  1997,  2014,  2048,
         4937,  3723, 27408,  1012,  5736,  2245,  2016,  3236,  2068,  4372,
         5210, 17884,  2063,  3972,  2594,  3406,  1012,  5736, 11618, 24401,
         1005,  1055,  6007,  2041,  1996,  2341,  1012, 24401, 11206,  3727,
         1010,  1998,  2059,  7719,  1999,  1996,  2690,  1997,  1996,  2346,
         2000,  2404,  2010,  6007,  2006,  1012,  2059,  2002,  4152,  2448,
         2058,  1006,  1000, 10560,  1000,  1010,  2028,  1997,  1996,  3574,
         1997,  1996,  2516,  1007,  2011,  1037,  4744,  1012,  1998,  8289,
         1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  10

### 2️⃣Positional encoding

In [6]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


**(B, S, D)**   
- Batch size: 한 번에 처리할 문장의 개수
- Sequence length: 한 문장 내의 토큰 개수
- Embedding Dimension: 각 토큰이 표현되는 벡터의 길이

example   
- 문장 3개를 한 번에 처리
- 각 문장이 최대 5개의 토큰을 가지고
- 각 토큰이 4차원 embedding 벡터로 표현된다면
``` python
x.shape = (3, 5, 4)  # (B, S, D)
```

<br>

**Positional encoding 마지막에 [None, ...]처리 이유**   
- `angle_rads.shape=(400, 256)`: (S, D)
- Transformer Model에 넣어야 하는 input은 (B, S, D) 형식이어야 함
- Broadcasting을 위해 (1, S, D)로 reshape 하는 것

###3️⃣Self-attention(Head-Attention)
- 이걸 병렬로 만들어서 사용하면 Multi-Head-Attention

In [7]:
q = torch.randn(1, 2, 3)  # shape: (B=1, S=2, D=3)
k = torch.randn(1, 2, 3)  # shape: (B=1, S=2, D=3)

# transpose K: (B, D, S)
k_trans = k.transpose(-1, -2)  # shape: (1, 3, 2)

# Attention score 계산
score = torch.matmul(q, k_trans)  # (1, 2, 3) @ (1, 3, 2) → (1, 2, 2)
print(score.shape)

torch.Size([1, 2, 2])


![self-attention](https://i.sstatic.net/4mhWz.png)

**shape 변화**   
```
x:            (B, S, input_dim)

Q, K, V:      (B, S, d_model)           ← Linear projection

Q @ Kᵀ:       (B, S, S)                 ← Attention score

Softmax:      (B, S, S)                 ← Attention weights

score @ V:    (B, S, d_model)           ← Weighted sum

Dense:        (B, S, d_model)           ← Output projection

```

<br>

**flow**   
```
[B, S, input_dim]
    │
    ├── Linear → Q (B, S, D)
    ├── Linear → K (B, S, D)
    └── Linear → V (B, S, D)
            ↓
      Q · Kᵀ → Attention Score (B, S, S)
            ↓
         Softmax
            ↓
      Attention × V → (B, S, D)
            ↓
         Dense
            ↓
      Output (B, S, D)

```

In [8]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    # Q, K, V weight를 선형 변환으로 (B, S, D) shape으로 만든다
    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)

    # dense 처리를 통해 사용자가 원하는 dimension으로 만든다
    self.dense = nn.Linear(d_model, d_model)

    # softmax를 통해 weight를 -1 ~ 1 사이의 값으로 만든다
    # dim=-1은 마지막 차원이라는 뜻
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)

    # softmax 전처리
    # score 값이 너무 커지면, softmax에서 gradient가 0에 가까워지는 vanishing gradient가 발생한다
    # 따라서 sqrt(d_model)으로 score를 모두 동일하게 나누어줌으로써 vanishing을 막고 안정적인 학습이 가능하게 도와준다
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    # Masked-Head-Attention 처리
    # -1e9을 곱해 음의 무한대 처리를 하여, 현재 값이 미래에 입력될 값에 대해 알지 못하도록 처리함
    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)

    # dense를 통해 result를 다시 선형변환 해줌
    # residual connection을 하기 위함임
    result = self.dense(result)

    return result

### 4️⃣Encoder Layer
- Self-Attention (Head Attention)
- Feed Forward Network(FFN)

**빠진 부분**
- Residual Connection: 입력값 + 출력값 처리
- Layer Normalization: 출력 안정화
- Dropout: 정규화 (optional)

In [9]:
class TransformerLayer(nn.Module):
  # input_dim: 입력 임베딩의 차원
  # d_model: Attention 내부에서 쓰는 차원
  # dff: FNN의 중간 차원(확장 차원)
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    # Self-Attention
    self.sa = SelfAttention(input_dim, d_model)

    # Feed Forward: Position-wise FFN
    # 각 토큰 위치별로 독립적으로 작
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  # x: 입력 문장 시퀀스 (B, S, input_dim)
  # mask: Attention mask (optional)
  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

### Transformer 기반 문장 분류기
- Embedding: 토큰 ID -> 벡터
- Positional Encoding: 순서 정보 부여
- N개의 Transformer Layer: 문맥 정보 반영
- Classification Head: `[CLS]` 위치에서 이진 분류 (sigmoid 전 단계)

In [10]:
class TextClassifier(nn.Module):
  # n_layers: Transformer 총 layer 수
  # dff: FFN 중간 차원 수
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    # Embedding
    self.embedding = nn.Embedding(vocab_size, d_model)

    # PE
    # requires_grad=False: 학습되지 않는 fixed encoding
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)

    # Encoder Stack Layer
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])

    # Classification Head
    # logit 출력
    self.classification = nn.Linear(d_model, 1)

  # x: [B, S]: 문장 길이 S의 배치 B개(토큰 ID)
  def forward(self, x):

    # Mask 생성: attention score에 사용할 padding mask
    # 해당 위치 무시되도록 처리 score + mask * -1e9
    mask = (x == tokenizer.pad_token_id) # (B, S), padding 위치 true
    mask = mask[:, None, :]              # (B, 1, S) - Broadcasting 위해 차원 확장
    seq_len = x.shape[1]

    # embedding + position encoding
    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    # Transformer layer stack
    for layer in self.layers:
      x = layer(x, mask)                # (B, S, D) shape 유지

    # [CLS] 위치 벡터 추출 & 분류
    x = x[:, 0]                         # (B, D) 첫 토큰의 벡터 사용
    x = self.classification(x)          # (B, 1) 이진 분류 결과(LInear)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [11]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [12]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [13]:
n_epochs = 5

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 10.657742949202657
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   1 | Train Loss: 0.018316637724637985
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   2 | Train Loss: 2.980232238769531e-07
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   3 | Train Loss: 4.470348358154297e-08
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   4 | Train Loss: 2.9802322387695312e-08
=========> Train acc: 1.000 | Test acc: 1.000
